In [1]:
from __future__ import print_function
import os, glob
import time
import numpy as np
import yaml
import pickle
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

from tensorboardX import SummaryWriter
from data.dataset import SkeletonDataset

In [3]:
skeleton = SkeletonDataset(root="/home/project/gcn/Apb-gcn/NTU-RGB+D", name='cv_train', benchmark='cv', sample = 'train')

In [3]:
directory = '/home/project/gcn/Apb-gcn/NTU-RGB+D/processed/cs_train.pt'

tt = torch.load(directory)

In [13]:
tt[1].shape

torch.Size([40091])

In [11]:
print(len(tt[0]), len(tt[1]))

40091 40091


In [8]:
tt[0][1].shape

torch.Size([94, 25, 7])

In [14]:
tf = '/home/project/gcn/Apb-gcn/NTU-RGB+D/test/test2.skeleton'

In [15]:
def process_skeleton(path,
                     num_joints=25,
                     num_features=3,
                     use_motion_vector=False):
    import os.path as osp
    #t = osp.split(path)[-1][-12:-9]
    t = 2
    print(path)
    with open(path, 'r') as f:
        lines = f.readlines()
        num_frames = int(lines[0])
        print("num_frames: ", num_frames)
        start = 1
        num_persons = int(lines[1])
        print("num_persons: ", num_persons)
        offset = int((len(lines) - 1) / num_frames)
        frames = [lines[start + 3 + i * offset:
                        start + 3 + i * offset + num_joints] for i in range(num_frames)]
        #print(frames[1])
        frames = process_frames(frames, num_joints, num_features, use_motion_vector)
        if num_persons == 2:
            frames_ = [lines[start + (i + 1) * offset - num_joints:
                             start + (i + 1) * offset + num_joints] for i in range(num_frames)]
            print(frames_[0])
            frames_ = process_frames(frames_, num_joints, num_features, use_motion_vector)
            frames = torch.cat([frames, frames_], dim=0)
        return frames, int(t)

In [16]:
def motion_vector(frames):
    # dimensions: num_frames num_joints num_features (x, y, z)
    mgd = torch.sqrt(torch.sum(torch.square(frames), dim=2))  # Magnitude
    mv = torch.zeros(frames.shape)
    mv[1:, :, :] = frames[1:, :, :] - frames[0: -1, :, :]
    mv = torch.div(mv.view(-1, 3), mgd.view(-1, 1)).view(frames.shape)
    # switch the order to {z, x, y} before applying acos
    mv = torch.cat([torch.acos(mv[:, :, [2, 0, 1]]),
                    torch.unsqueeze(mgd, dim=2)], dim=-1)
    return mv

In [ ]:
[ff[index] for index in range(len(ff)) if index % 3 == 2]

In [17]:
def process_frames(frames, num_joints, num_features, use_motion_vector=False):
    fv = torch.zeros((len(frames), num_joints, num_features))
    for i in range(len(frames)):
        f = frames[i]
        for j in range(num_joints):
            vs = [float(n) for n in f[j].split()][0: num_features]
            fv[i, j, :] = torch.tensor(vs)
    if use_motion_vector:
        fv = torch.cat([fv, motion_vector(fv)], dim=-1)
    return fv

In [18]:
def updated_process_frames(frames, num_persons, num_joints, num_features, use_motion_vector=False):
    fv = torch.zeros((len(frames), num_joints, num_features))
    frame_count = 0
    for p in range(num_persons):
        per_person = [frames[index] for index in range(len(frames)) if index % num_persons == p]
        for i in range(len(per_person)):
            f = per_person[i]
            for j in range(num_joints):
                vs = [float(n) for n in f[j].split()][0: num_features]
                fv[frame_count, j, :] = torch.tensor(vs)
            frame_count += 1
    if use_motion_vector:
        fv = torch.cat([fv, motion_vector(fv)], dim=-1)
    return fv

In [19]:
def updated_process_skeleton(path,
                     num_joints=25,
                     num_features=3,
                     use_motion_vector=True):
    import os.path as osp
    #t = osp.split(path)[-1][-12:-9]
    t = 2
    count = 0
    print(path)
    with open(path, 'r') as f:
        i = 0
        lines = f.readlines()
        frames = []
        while i < len(lines):
            #print(i)
            if i == 0:
                frame_number = lines[i]
                i += 1
            else:
                if lines[i] == '0\n':
                    i += 1
                    continue
                num_persons = int(lines[i])
                #num_joints = int(lines[i+2])
                for j in range(num_persons):
                    frames.append(lines[i+3+j*27:i+28+j*27])
                i += (1 + num_persons * 27)
                    
        frames = updated_process_frames(frames, num_persons, num_joints, num_features, use_motion_vector)            
    return frames, int(t)

In [20]:
f, idx = process_skeleton(tf)

/home/project/gcn/Apb-gcn/NTU-RGB+D/test/test2.skeleton
num_frames:  3
num_persons:  2
['1.143176 0.7084895 4.362553 352.7569 148.7456 1250.743 396.291 -0.0676174 0.004678101 0.9698148 -0.2342334 2\n', '1.136982 0.95244 4.23788 355.1589 125.7225 1257.953 330.336 -0.06834885 0.004854767 0.9697635 -0.2342298 2\n', '1.127422 1.191693 4.103536 357.6232 101.4471 1265.468 260.7529 -0.07128434 -0.00520127 0.960898 -0.2675193 2\n', '1.115176 1.28857 4.029669 358.3796 90.59951 1267.951 229.5862 0 0 0 0 2\n', '0.9758932 1.100306 4.172343 342.4962 111.3647 1221.891 288.9844 0.2191297 0.765861 -0.5944641 0.1097793 2\n', '0.9305833 0.9081436 4.31139 335.7446 131.0136 1202.17 345.1609 0.1695074 0.726771 -0.2708969 -0.6080181 2\n', '0.8996224 0.7457207 4.393685 331.5867 146.1283 1190.062 388.4174 0.1962227 0.9302434 -0.147619 -0.2726766 2\n', '0.9127301 0.7226607 4.402908 332.5195 148.1863 1192.713 394.3375 0.2571706 0.870483 0.1526045 -0.3909406 1\n', '1.297338 1.077064 4.133075 372.0637 112.4272 13

In [14]:
f.shape

torch.Size([6, 25, 3])

In [11]:
idx

2

In [21]:
f, idx = updated_process_skeleton(tf)

/home/project/gcn/Apb-gcn/NTU-RGB+D/test/test2.skeleton


In [22]:
f.shape

torch.Size([6, 25, 7])

In [23]:
idx

2

In [103]:
f[0]

tensor([[ 5.3362e-01,  7.6619e-02,  3.6079e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.6480e+00],
        [ 5.1281e-01,  3.4690e-01,  3.4416e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.4969e+00],
        [ 4.9054e-01,  6.1125e-01,  3.2656e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.3584e+00],
        [ 4.8491e-01,  7.0598e-01,  3.2735e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.3837e+00],
        [ 3.4536e-01,  5.2826e-01,  3.3594e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.4182e+00],
        [ 4.3483e-01,  3.2148e-01,  3.4627e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.5046e+00],
        [ 4.6799e-01,  1.0203e-01,  3.6515e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.6828e+00],
        [ 4.4591e-01, -3.8013e-03,  3.6529e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.6801e+00],
        [ 5.7847e-01,  4.6453e-01,  3.2299e+00,  1.5708e+00,  1.5708e+00,
          1.5708e+00,  3.3140e+00],
        [ 

In [2]:
directory = '/home/project/gcn/Apb-gcn/NTU-RGB+D/processed/cs/cs_val.pt'

tt = torch.load(directory)


In [4]:
print(tt[0][0].shape, tt[1].shape)

torch.Size([98, 25, 7]) torch.Size([70256])


In [24]:
len(tt[0])

43689

In [20]:
print(len(tt[0]), len(tt[1]))

70256 70256


In [62]:
#tt[1]

In [63]:
#tl = torch.zeros(3)
#tl[0] = 1
#tl[1] = 2
#tl[2] = 3

#print(tl)


In [64]:
#ll = [1,2,3]
#bb = torch.FloatTensor(ll)
#print(bb)

In [54]:
ff=[1,2,3,4,5,6,7,8,9]

In [58]:
[ff[index] for index in range(len(ff)) if index % 3 == 2]

[3, 6, 9]